# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier as a reward function.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets the start of a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

## %pip install transformers trl wandb


In [3]:

## Python >= 3.8  


In [4]:

## !pip install transformers
## !pip install wandb
## !pip install pandas
## !pip install datasets
## !pip install accelerate
## !pip install tyro


In [5]:
import torch
from tqdm import tqdm
import pandas as pd
import wandb
import os

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

### Configuration

In [6]:

config = PPOConfig(
    model_name    = "lvwerra/gpt2-imdb",
    learning_rate = 1.41e-5,
    ## log_with      = "wandb",
)

sent_kwargs = {
         "return_all_scores": True, 
         "function_to_apply": "none", 
         "batch_size": 16
}


In [7]:

## wandb.init()

wandb.init(mode="disabled") 
os.environ['WANDB_DISABLED'] = 'true'



You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [8]:

def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    # load imdb with datasets
    
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode( sample["review"]    )[: input_size()]
        sample["query"]     = tokenizer.decode( sample["input_ids"] )
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds
    

In [9]:

dataset = build_dataset(config)


def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])
    

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [10]:
model     = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token


### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [11]:

ppo_trainer = PPOTrainer(
                   config, 
                   model, 
                   ref_model, 
                   tokenizer, 
                   dataset       = dataset, 
                   data_collator = collator
)


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [12]:

device = ppo_trainer.accelerator.device

if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
    
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)


The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [13]:

text = "this movie was really bad!!"

sentiment_pipe(text, **sent_kwargs)


/home/maquina1/anaconda3/envs/py38_RLHF/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.3350484371185303},
  {'label': 'POSITIVE', 'score': -2.726576328277588}]]

In [14]:

text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)


[[{'label': 'NEGATIVE', 'score': -2.294790267944336},
  {'label': 'POSITIVE', 'score': 2.557040214538574}]]

### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [15]:

gen_kwargs = {
         "min_length":   -1, 
         "top_k":       0.0, 
         "top_p":       1.0, 
         "do_sample":  True, 
         "pad_token_id": tokenizer.eos_token_id
}


## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [16]:

output_min_length     = 4
output_max_length     = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)



In [17]:

generation_kwargs = {
    "min_length":     -1,
    "top_k":         0.0,
    "top_p":         1.0,
    "do_sample":    True,
    "pad_token_id": tokenizer.eos_token_id,
}


In [18]:

## ppo_trainer.config.steps = 100    ## 20,000
ppo_trainer.config.steps


20000

In [19]:
'''
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    print(batch)
    input()
'''

'\nfor epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):\n    print(batch)\n    input()\n'

In [20]:


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(
                     query_tensors, 
                     response_tensors, 
                     rewards
    )
    ppo_trainer.log_stats(stats, batch, rewards)
    

0it [00:00, ?it/s]

0


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (14.16) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (19.34) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (22.25) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (22.97) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average 

1


/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (17.44) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (20.91) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (11.10) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (11.14) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (15.86) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (10.33) exceeds threshold 10.00. Skippin

...

/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (34.87) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (155.94) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (25.60) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (6172.97) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (124.20) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (14.71) exceeds threshold 10.00. Ski

95


/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (41.81) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (35.72) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (31.05) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (99.97) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (407.93) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (11.88) exceeds threshold 10.00. Skippi

96


/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (286.39) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (40.03) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (25.03) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (46.02) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (510.09) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/maquina1/Desktop/RLHF/trl/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (49.43) exceeds threshold 10.00. Skipp

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://app.wandb.ai/huggingface/trl-showcase/runs/1jtvxb1m/).

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `model_ref` to compare the tuned model `model` against the model before optimisation.

In [21]:
#### get a batch from the dataset
bs                 = 16
game_data          = dict()
dataset.set_format("pandas")
df_batch           = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors      = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

/home/maquina1/anaconda3/envs/py38_RLHF/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/home/maquina1/anaconda3/envs/py38_RLHF/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,When originally screened in America in 1972,", a ""Fare",", the first closer to",0.518275,1.927933
1,Born Again,fiction that has inspired many productions of...,is a very funny look of great originality and...,2.276586,2.823155
2,"Generally speaking, the plot",moves into why Dr. Byng and his eight,is clever. It's a great one and has,1.093851,2.788485
3,I have seen the recent,"movies of much greater talent (Or, yes","episodes play out now, and I is always",1.259309,2.337726
4,"Description: Corny,",with a bit of flair in voice-,truly stunning. Co-stared and,-0.994446,1.730018
5,"Director Edward Sedgwick,","who can use all the characters, mainly of whom","here, I love films that honor a strong cast",1.231162,2.489708
6,Robert Siodmak does,excellent pointing out of Dwight McBride. Dan...,"a great job. He is a talented boy, and",1.886305,2.482173
7,DO NOT WATCH THIS,MOVIE). I had the privilege,!! It was very well worth it,-0.750461,2.693946
8,Where to,get it: I forget if they inked Warren Beatty ...,Where to contact it is a masterpiece!<|endofte...,-1.792121,2.458855
9,STAR RATING: ***** The,"movie starts out perfect and cool, it can sti...","best Writing in Cinema, which has a great cas...",1.901080,2.448417


Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [22]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    0.379524
rewards (after)     2.441947
dtype: float64


median:


rewards (before)    1.115460
rewards (after)     2.485941
dtype: float64

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [24]:

## model.save_pretrained(    "gpt2-imdb-pos-v2", push_to_hub=True)
## tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)

model.save_pretrained(    "gpt2-imdb-pos-v2", push_to_hub=False)
tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=False)


('gpt2-imdb-pos-v2/tokenizer_config.json',
 'gpt2-imdb-pos-v2/special_tokens_map.json',
 'gpt2-imdb-pos-v2/vocab.json',
 'gpt2-imdb-pos-v2/merges.txt',
 'gpt2-imdb-pos-v2/added_tokens.json',
 'gpt2-imdb-pos-v2/tokenizer.json')

In [25]:

print(tqdm)


<class 'tqdm.std.tqdm'>
